In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob,os
import sys
import scipy
from importlib import  reload
from time import process_time 
#from libraries.lib_gather_data import get_hhid_FIES

%load_ext autoreload

In [2]:
from shock_libraries import *
from plotting_libraries import *
from response_libraries import get_response_sp
#
from income_shock_libraries_ps import *
#
from libraries.lib_country_dir import set_directories, load_survey_data, get_places_dict
from libraries.lib_get_hh_savings import get_hh_savings
from libraries.pandas_helper import broadcast_simple

In [3]:
# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

In [9]:
## create crosstabs for sectors for K:

# load file:
df = pd.read_csv('./csv/lfs_a09_pqkb_ranked_20200420.csv')
df

#pd.crosstab(s1, b1, dropna=False)
df_xtab = pd.crosstab(df['a09_pqkb'], df['sector'], dropna=False)
df_xtab.to_csv('./temp/sector_crosstab_20200420.csv')

In [11]:
## create crosstabs for sectors for K:


mr = merge_rank()

    # get subset: a09_pqkb
df = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home']]

# load file:
#df = pd.read_csv('./csv/lfs_a09_pqkb_ranked_20200420.csv')
#df

#pd.crosstab(s1, b1, dropna=False)
df_xtab = pd.crosstab(df['c19_pclass'], df['LFS_sector'], dropna=False)
df_xtab.to_csv('./temp/sector_c19_crosstab_20200420.csv')

In [4]:

def generate_shock_100():  # initialize shock sector storage dataframe
   
    '''
   current hard coding for sensitivity analysis, 20200413: requires cleaning for further implementation
   - addition of modularity
   - 
   - current functionality:
       - outputs csv to location: './temp/sect_iter_100.csv
       - containing data frame with 101 simulations of <rand_weighted_shock_distance():
    - runtime: ~10minutes
   '''

    stor = rand_weighted_shock_distance()
    del stor['di']

    # set number of iterations
    p = 0
    n_iter = 99

    # model and store stochastic sector response
    while p < n_iter:
        new_val = rand_weighted_shock_distance()
        del new_val['di']
        new_val = new_val.rename(columns={'fa': ('iter'+str(p))})

        # pd.merge(labor,rank, on=merge_col, how='left')
        stor = pd.merge(stor,new_val,on='LFS_sector', how='left')
        p = p+ 1
        print(p)
    stor.to_csv('./temp/sect_iter_3d_100_20200420.csv')



In [5]:
## modified function to include other income streams:

def adjust_income_and_weight_all_wage(df,useLFS=True):
    wgt = get_weight_handle(useLFS)
    df['wage_loss'] = 0
    
           
    # this is for bifurcated (a/na) setup
    #df.loc[df.eval("affected_cat=='na'"),wgt+'_shock'] = df.loc[df.eval("affected_cat=='na'"),wgt].copy()
    #wgt_sanity_check(df,wgt=wgt)
    #_a = "(affected_cat=='a')&(cempst1=='Employed')"
        
        
    df['affected'] = 0
        
    # ex: fa = 0.8 --> 80% chance rand < fa --> affected = True
    is_affected = df['fa']>np.random.uniform(0,1,df.shape[0])
    df.loc[is_affected,'affected'] = 1
        
    # not employed = not affected
    not_employed = "(cempst1!='Employed')"
    df.loc[df.eval(not_employed),'affected'] = 0
    
    # ^ affected includes employment (redundant)
    # frac_hours_worked & di = 0 for non-workers
    # nonagri_sal is constant for entire household
    
    df['wage_loss'] = df[['affected','frac_hours_worked','di','nonagri_sal']].prod(axis=1)
    # income loss ANNUAL here
    #
    return df

In [6]:
### 20200419 -- Incorporate 3rd factor into sectoral shock



In [7]:
rand_weighted_shock_3dim()

/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/psaylor/Documents/GitHub/covid_phl/income_shock_libraries_ps.py:596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mr_subset.partial_prob[i] = mr_subset.sector_frac[i] * (random.randint(0,50)/100)
/Users/psaylor/Documents/GitHub/covid_phl/income_shock_libraries_ps.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,fa,di
LFS_sector,,
ag,0.700497,0.0
mining,0.972295,0.0
utilities,0.629517,0.0
construction,0.908082,1.0
manufacturing,0.947248,1.0
wholesale,0.916176,1.0
retail,0.898754,1.0
transportation,0.973561,1.0
information,0.709419,1.0
